<a href="https://colab.research.google.com/github/tanyaachaturvedi/Sparse_RBM/blob/main/Sparse_RBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [4]:
from google.colab import files
files.upload()



Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"tanyaachaturvedi","key":"b3c75bbea56ad31f1033d54fa3fc6816"}'}

In [6]:
!mv "kaggle.json" kaggle.json  # rename it if needed
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json



mv: 'kaggle.json' and 'kaggle.json' are the same file


In [7]:
# Download dataset
!kaggle datasets download -d crawford/gene-expression --force

# Unzip
import zipfile
with zipfile.ZipFile("gene-expression.zip", "r") as zip_ref:
    zip_ref.extractall("gene_expression_data")

Dataset URL: https://www.kaggle.com/datasets/crawford/gene-expression
License(s): CC0-1.0
  0% 0.00/1.41M [00:00<?, ?B/s]
100% 1.41M/1.41M [00:00<00:00, 727MB/s]


In [8]:
!ls gene_expression_data


actual.csv  data_set_ALL_AML_independent.csv  data_set_ALL_AML_train.csv


In [9]:
import pandas as pd

# Load all CSVs
actual_df = pd.read_csv("gene_expression_data/actual.csv")
train_df = pd.read_csv("gene_expression_data/data_set_ALL_AML_train.csv")
test_df = pd.read_csv("gene_expression_data/data_set_ALL_AML_independent.csv")

# Quick preview
print("Actual:")
print(actual_df.head())
print("\nTrain shape:", train_df.shape)
print("Test shape:", test_df.shape)


Actual:
   patient cancer
0        1    ALL
1        2    ALL
2        3    ALL
3        4    ALL
4        5    ALL

Train shape: (7129, 78)
Test shape: (7129, 70)


In [10]:
# Combine train and test
full_df = pd.concat([train_df, test_df], ignore_index=True)

# Merge labels into full_df
full_df["cancer"] = actual_df["cancer"]

# Drop non-feature columns if any
X = full_df.drop(columns=["cancer"])
y = full_df["cancer"]


In [12]:
# Set gene names as index and transpose to get samples as rows
full_df.set_index(full_df.columns[0], inplace=True)
X = full_df.drop(columns=["cancer"]).T  # transpose and drop label column
X = X.apply(pd.to_numeric, errors='coerce')  # ensure numeric

# Re-attach the labels
y = actual_df["cancer"].values

# Now normalize
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Standardizing the dataset

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
scaler = StandardScaler()

In [17]:
import pandas as pd

# Load files
actual_df = pd.read_csv("gene_expression_data/actual.csv")
train_df = pd.read_csv("gene_expression_data/data_set_ALL_AML_train.csv")
test_df = pd.read_csv("gene_expression_data/data_set_ALL_AML_independent.csv")

# Combine expression data
expression_df = pd.concat([train_df, test_df], axis=1)


In [18]:
# First row in expression_df is gene names — set it as index
expression_df = expression_df.set_index(expression_df.columns[0])

# Transpose so that each row = 1 patient, each column = 1 gene
X = expression_df.T

# Ensure all values are numeric
X = X.apply(pd.to_numeric, errors='coerce')

In [19]:
from sklearn.preprocessing import LabelEncoder

# Get cancer types for each patient
y = actual_df['cancer']
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Normalizing

In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [21]:
print("Features shape:", X_scaled.shape)
print("Labels shape:", y_encoded.shape)


Features shape: (146, 7129)
Labels shape: (72,)


In [23]:
print(X.shape)


(146, 7129)


# Applying PCA

In [27]:
from sklearn.impute import SimpleImputer

# Fill NaNs with column (gene) means
imputer = SimpleImputer(strategy='mean')
X_scaled_imputed = imputer.fit_transform(X_scaled)


In [28]:
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_scaled_imputed)


In [30]:
from sklearn.impute import SimpleImputer


In [31]:
imputer = SimpleImputer(strategy='mean')
X_scaled_imputed = imputer.fit_transform(X_scaled)


In [32]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_scaled_imputed)


In [33]:
explained = np.sum(pca.explained_variance_ratio_)
print(f"Total variance explained by 50 components: {explained:.2%}")


Total variance explained by 50 components: 89.64%
